# 📐 Module 2 — Mathematical Model
## Image Noise & Gaussian Filtering Project

---

> **Objective:** Understand the Gaussian (Normal) distribution, its parameters — mean (μ) and variance (σ²) — and derive the noise equation used to model and simulate image noise.

**Sections in this notebook:**
1. What is a Probability Distribution?
2. The Gaussian (Normal) Distribution & PDF
3. Mean (μ) — The Center of Noise
4. Variance (σ²) and Standard Deviation (σ)
5. The 68–95–99.7 Rule
6. The Gaussian Noise Equation
7. Summary

---

## 1. What is a Probability Distribution?

In real-world imaging sensors, every pixel measurement contains a small **random error**. These errors come from thermal fluctuations in sensor electronics, photon shot noise, and electronic readout noise.

These errors are **random** — but not completely shapeless. They follow a pattern:
- Small errors are **common**
- Large errors are **rare**
- Very large errors are **extremely rare**

A **probability distribution** is a mathematical function that describes how likely different noise values are to occur. Instead of saying *'the noise can be anything'*, a distribution gives shape and structure to the randomness.

> 💡 **Key Intuition:** A distribution tells us where values tend to cluster and how spread out they are. The Gaussian distribution says: errors near zero are most common, and the further from zero, the rarer they become.

---
## 2. The Gaussian (Normal) Distribution

The **Gaussian distribution** — also called the **Normal distribution** — is the most widely used probability distribution in science and engineering. It produces the famous **bell-shaped curve**.

It is completely defined by just **two parameters**:

| Parameter | Symbol | Role |
|-----------|--------|------|
| Mean | μ (mu) | The center / peak of the bell curve |
| Variance | σ² (sigma squared) | The width / spread of the bell curve |

---

### 2.1 The Probability Density Function (PDF)

The shape of the Gaussian is given by its **Probability Density Function (PDF)**. This function tells us: *for a given noise value $x$, how likely is that value to occur?*

$$
f(x \mid \mu, \sigma^2) = \frac{1}{\sigma\sqrt{2\pi}} \; e^{\displaystyle -\frac{(x - \mu)^2}{2\sigma^2}}
$$

---

### 2.2 Breaking Down the Formula

| Component | Symbol | Meaning |
|-----------|--------|---------|
| Random variable | $x$ | The specific noise value being evaluated (e.g. a pixel error of +5) |
| Mean | $\mu$ | The center of the distribution. For image noise: usually $\mu = 0$ |
| Variance | $\sigma^2$ | Controls the spread. Large $\sigma^2$ → wide, flat curve → more intense noise |
| Standard deviation | $\sigma$ | Square root of variance. Same units as $x$ (e.g. pixel intensity) |
| Euler's number | $e \approx 2.718$ | Creates the smooth, exponentially decaying tails of the curve |
| Normalization factor | $\frac{1}{\sigma\sqrt{2\pi}}$ | Ensures total area under the curve = 1 (total probability = 100%) |
| Exponent | $-\frac{(x-\mu)^2}{2\sigma^2}$ | Measures how far $x$ is from the mean, scaled by the spread |

> 📌 **Note:** The exponent is always **negative**, so as $x$ moves further from $\mu$, the exponent becomes more negative, and $e^{\text{negative number}}$ shrinks toward zero — this is what creates the tails of the bell curve.

---
## 3. Mean (μ) — The Center of Noise

The **mean** $\mu$ is the average value around which noise fluctuates. It sits at the exact **peak** of the bell curve — the most likely value to be observed.

$$
\mu = \frac{1}{N} \sum_{i=1}^{N} n_i
$$

where $n_i$ are individual noise samples and $N$ is the total number of samples.

---

### 3.1 Why μ = 0 for Image Noise

In image processing, Gaussian noise is almost always modeled with **zero mean** ($\mu = 0$). This reflects a physical reality:

- Sensor noise doesn't systematically **brighten** or **darken** pixels
- It randomly fluctuates **upward and downward in equal measure**
- On average, no net brightness is added to the image — only random scatter

> 💡 **Example:** If $\mu = 0$ and $\sigma = 25$, then a noise value of $+15$ and a noise value of $-15$ are equally probable. The image is randomly corrupted, but not biased toward brightness or darkness.

---

### 3.2 Effect of Changing μ

| Mean (μ) | Curve Position | Practical Effect on Image |
|----------|---------------|---------------------------|
| $\mu = 0$ | Centered at zero | Noise adds and subtracts equally — unbiased |
| $\mu > 0$ (e.g. +30) | Shifted right | Noise systematically **brightens** pixels |
| $\mu < 0$ (e.g. −30) | Shifted left | Noise systematically **darkens** pixels |

---
## 4. Variance (σ²) and Standard Deviation (σ)

**Variance** $\sigma^2$ measures how widely the noise values are spread around the mean. It is the most important parameter for controlling **noise intensity** in simulations.

$$
\sigma^2 = \frac{1}{N} \sum_{i=1}^{N} (n_i - \mu)^2
$$

**Standard deviation** $\sigma$ is the square root of variance:

$$
\sigma = \sqrt{\sigma^2}
$$

Standard deviation is expressed in the **same units as the pixel values** (0–255 for 8-bit images), making it more intuitive to interpret than variance.

---

### 4.1 Why We Square in the Variance Formula

The deviations $(n_i - \mu)$ can be positive or negative. If we simply averaged them, positive and negative errors would cancel each other out to zero. By **squaring** them:
- Both positive and negative errors contribute **positively** to variance
- Larger errors are weighted **more heavily**
- The result is a true measure of average spread

---

### 4.2 Visualizing Variance

> 💡 **Analogy:** Think of variance as the energy of the noise.
> - **Low σ²** → gentle ripples on a calm lake → slight graininess in the image
> - **High σ²** → violent storm waves → heavy noise, image details obscured

| Noise Level | Std. Dev. (σ) | Variance (σ²) | Visual Impact |
|-------------|--------------|---------------|---------------|
| Very Low    | σ = 5        | σ² = 25       | Barely perceptible |
| Low         | σ = 15       | σ² = 225      | Slight graininess |
| Medium      | σ = 25       | σ² = 625      | Clearly visible noise |
| High        | σ = 50       | σ² = 2,500    | Heavy noise, details lost |
| Very High   | σ = 100      | σ² = 10,000   | Image nearly unrecognizable |

---

### 4.3 How σ Shapes the Bell Curve

The standard deviation $\sigma$ directly controls the **width** of the bell curve:

- **Small $\sigma$** → tall, narrow peak → noise values tightly clustered around the mean
- **Large $\sigma$** → short, wide, flat curve → noise values spread far from the mean

Both curves integrate to the same total area = 1, since probability must always sum to 100%.

---
## 5. The 68–95–99.7 Rule (Empirical Rule)

One of the most useful properties of the Gaussian distribution is the **empirical rule**, which tells us exactly how noise values are distributed relative to the standard deviation:

$$
P(\mu - \sigma \leq x \leq \mu + \sigma) \approx 68.27\%
$$
$$
P(\mu - 2\sigma \leq x \leq \mu + 2\sigma) \approx 95.45\%
$$
$$
P(\mu - 3\sigma \leq x \leq \mu + 3\sigma) \approx 99.73\%
$$

| Range | % of Noise Values | Meaning |
|-------|-------------------|---------|
| μ ± 1σ | 68.27% | ~2 in 3 noise samples fall within one std. dev. of the mean |
| μ ± 2σ | 95.45% | ~19 in 20 noise samples fall within two std. devs. |
| μ ± 3σ | 99.73% | Nearly all noise samples fall within three std. devs. |

> 📌 **Practical Example:** If $\mu = 0$ and $\sigma = 25$:
> - **68%** of noise values added to pixels will be between **−25 and +25**
> - **95%** will be between **−50 and +50**
> - Only **0.27%** of pixels receive noise beyond **±75**

---
## 6. The Gaussian Noise Equation

Now we bring everything together into the **complete noise model** used in image processing.

---

### 6.1 Additive Noise Model

Gaussian noise is modeled as **additive** — the noise value is directly added on top of the clean pixel:

$$
I_{\text{noisy}}(x, y) = I_{\text{original}}(x, y) + n(x, y)
$$

| Symbol | Meaning |
|--------|---------|
| $I_{\text{noisy}}(x, y)$ | Observed (corrupted) pixel intensity at position $(x, y)$ |
| $I_{\text{original}}(x, y)$ | True (clean) pixel intensity at position $(x, y)$ |
| $n(x, y)$ | Noise value at pixel $(x, y)$, sampled from a Gaussian distribution |

---

### 6.2 The Noise Term

The noise $n(x, y)$ at each pixel is **independently** sampled from the Gaussian distribution:

$$
n(x, y) \sim \mathcal{N}(\mu, \sigma^2)
$$

The symbol $\sim$ means **'is drawn from'** or **'follows the distribution of'**. Each pixel gets a **fresh, independent** random noise sample — no two pixels share the same noise value.

---

### 6.3 Complete Gaussian Noise Model

Combining both equations gives us the full model:

$$
\boxed{I_{\text{noisy}}(x, y) = I_{\text{original}}(x, y) + \mathcal{N}(\mu,\, \sigma^2)}
$$

> 📌 **Standard Assumption:** Set $\mu = 0$ (zero-mean, unbiased noise). The only tuning parameter is $\sigma^2$ — increasing it makes the image noisier.

---

### 6.4 Pixel Clipping

Pixel intensities must stay within the valid range **[0, 255]** for 8-bit images. After adding noise, we apply clipping:

$$
I_{\text{noisy}}(x, y) = \text{clip}\!\left(I_{\text{original}}(x, y) + n(x, y),\; 0,\; 255\right)
$$

Without clipping:
- Bright pixel (e.g. 240) + positive noise (+30) → **270** → overflow ❌
- Dark pixel (e.g. 10) + negative noise (−25) → **−15** → underflow ❌

With clipping, both are snapped back to the valid [0, 255] range ✅

---

### 6.5 Why Gaussian Noise is a Good Model

| Reason | Explanation |
|--------|-------------|
| **Central Limit Theorem** | Real sensor noise is the sum of many independent effects. By the CLT, their sum converges to a Gaussian — by mathematical law |
| **Separability** | A 2D Gaussian filter splits into two 1D passes, making it computationally efficient |
| **Mathematical tractability** | Gaussian distributions have clean, closed-form formulas easy to analyze and compute |
| **Maximum entropy** | For a fixed mean and variance, the Gaussian makes the fewest assumptions — the most natural model under uncertainty |

---
## 7. Summary

| Concept | Symbol | Formula | Role |
|---------|--------|---------|------|
| Mean | $\mu$ | $\mu = \frac{1}{N}\sum n_i$ | Center of noise distribution. Set to 0 for unbiased noise |
| Variance | $\sigma^2$ | $\sigma^2 = \frac{1}{N}\sum(n_i - \mu)^2$ | Controls noise intensity. Higher = noisier image |
| Std. Deviation | $\sigma$ | $\sigma = \sqrt{\sigma^2}$ | Spread in pixel units. Most intuitive noise parameter |
| PDF | $f(x)$ | $\frac{1}{\sigma\sqrt{2\pi}}e^{-\frac{(x-\mu)^2}{2\sigma^2}}$ | Probability of any specific noise value occurring |
| Noise sample | $n(x,y)$ | $n(x,y) \sim \mathcal{N}(\mu, \sigma^2)$ | Random noise drawn independently per pixel |
| Full noise model | $I_{\text{noisy}}$ | $I_{\text{orig}} + \mathcal{N}(\mu, \sigma^2)$ | Noise is directly added to the clean pixel value |
| With clipping | $I_{\text{noisy}}$ | $\text{clip}(I_{\text{orig}} + n,\; 0,\; 255)$ | Keeps pixel values within valid 8-bit range |

---

### Key Takeaways

- The Gaussian distribution is defined entirely by **μ** (center) and **σ²** (spread)
- For image noise, we use **μ = 0** — noise is symmetric and unbiased
- **σ** (standard deviation) is the practical control knob: higher σ = more visible noise
- The **68–95–99.7 rule** lets you predict exactly how much noise a pixel is likely to receive
- Noise is **additive and independent per pixel**: $I_{\text{noisy}} = I_{\text{original}} + \mathcal{N}(0, \sigma^2)$
- Always **clip** pixel values to [0, 255] after adding noise

---

> ➡️ **Next: Section 3 — Noise Simulation**
> We will use these equations in Python with `numpy.random.normal(mu, sigma, size)` to add Gaussian noise to real images and visualize the results side by side.